In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
def MinMaxScalar(data):
    bunja = data-np.min(data,0)
    bunmo = np.max(data,0) - np.min(data,0)
    return bunja/(bunmo+1e-10)

# 연습문제 워밍업

In [3]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
              [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
              [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
              [816, 820.958984, 1008100, 815.48999, 819.23999],
              [819.359985, 823, 1188100, 818.469971, 818.97998],
              [819, 823, 1198100, 816, 820.450012],
              [811.700012, 815.25, 1098100, 809.780029, 813.669983],
              [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

In [4]:
x_data.shape

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\mook\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-5190a6264e53>", line 1, in <module>
    x_data.shape
NameError: name 'x_data' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mook\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mook\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\mook\Anaconda3\lib\site-packages\IPython\core\ultratb.py

NameError: name 'x_data' is not defined

In [ ]:
xy= MinMaxScalar(xy)

In [ ]:
x_data = xy[:,:-1]
y_data = xy[:,[-1]]
X= tf.placeholder(tf.float32, shape=[None,4])
Y= tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([4,1]))
B = tf.Variable(tf.random_normal([1]))

hypothesis = tf.matmul(X,W)+B
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

train = optimizer.minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2000):
    cost_v, _ = sess.run([cost, train], feed_dict={X:x_data,Y:y_data})
    if step %100 ==0 :
        print('step', step, 'cost',cost_v)

# MNIST 시작

In [ ]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.train.images)

In [11]:
x_train = mnist.train.images[:,:]
y_train = mnist.train.labels[:,:]

In [12]:
x_test = mnist.test.images[:,:]
y_test = mnist.test.labels[:,:]

In [ ]:
X= tf.placeholder(tf.float32, shape=[None,28*28*1])
Y= tf.placeholder(tf.float32, shape=[None,10])

W = tf.Variable(tf.random_normal([28*28*1,10]))
B = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(X, W) + B
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

### accuracy
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(1000):
    cost_v, _ = sess.run([cost, optimizer], feed_dict={X:x_train,Y:y_train})
    if step %100 ==0 :
        print('step', step, 'cost',cost_v)
print(sess.run(accuracy, feed_dict={X:x_train,Y:y_train}))

In [ ]:
print(sess.run(accuracy, feed_dict={X:x_test,Y:y_test}))

# 나워서 하기 training_epoch

In [ ]:
training_epoch = 15
batch_size = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) #550
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer],feed_dict={X:batch_xs,Y:batch_ys})
        avg_cost += c/total_batch
    print("Epoch :",epoch, 'Cost', avg_cost)

In [ ]:
print(sess.run(accuracy, feed_dict={X:x_test,Y:y_test}))

# 아무거나 뽑아서 확인해보기

In [ ]:
r = random.randint(0,mnist.test.num_examples-1)
print('Lbael:', sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print('Lbael:', sess.run(tf.argmax(hypothesis,1),feed_dict={X:mnist.test.images[r:r+1]}))
plt.imshow(
mnist.test.images[r:r+1].reshape(28,28),
    cmap='Greys',
    interpolation = 'nearest'
)

## 딥러닝으로 구현하기

In [ ]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.train.images)

In [ ]:
x_train = mnist.train.images[:,:]
y_train = mnist.train.labels[:,:]

In [ ]:
x_test = mnist.test.images[:,:]
y_test = mnist.test.labels[:,:]

In [ ]:
X= tf.placeholder(tf.float32, shape=[None,28*28*1])
Y= tf.placeholder(tf.float32, shape=[None,10])


W1 = tf.Variable(tf.random_normal([28*28*1,28*28*1]))
B1 = tf.Variable(tf.random_normal([28*28*1]))
layer1 = tf.sigmoid(tf.matmul(X,W1)+B1)

W2 = tf.Variable(tf.random_normal([28*28*1,28*28*1]))
B2 = tf.Variable(tf.random_normal([28*28*1]))
layer2 = tf.sigmoid(tf.matmul(layer1,W2)+B2)

W3 = tf.Variable(tf.random_normal([28*28*1,10]))
B3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer2, W3)+B3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

###
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# for step in range(1000):
#     cost_v, _ = sess.run([cost, optimizer], feed_dict={X:x_train,Y:y_train})
#     if step %100 ==0 :
#         print('step', step, 'cost',cost_v)
# print(sess.run(accuracy, feed_dict={X:x_train,Y:y_train}))

In [ ]:
training_epoch = 5
batch_size = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) #550
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer],feed_dict={X:batch_xs,Y:batch_ys})
        avg_cost += c/total_batch
    print("Epoch :",epoch, 'Cost', avg_cost)

In [ ]:
result = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
result

# 튜닝해보기
## sigmoid -> relu
## gradientdecent -> adamoptimizer

In [ ]:
X= tf.placeholder(tf.float32, shape=[None,28*28*1])
Y= tf.placeholder(tf.float32, shape=[None,10])


W1 = tf.Variable(tf.random_normal([28*28*1,28*28*1]))
B1 = tf.Variable(tf.random_normal([28*28*1]))
layer1 = tf.nn.relu(tf.matmul(X,W1)+B1)

W2 = tf.Variable(tf.random_normal([28*28*1,28*28*1]))
B2 = tf.Variable(tf.random_normal([28*28*1]))
layer2 = tf.nn.relu(tf.matmul(layer1,W2)+B2)

W3 = tf.Variable(tf.random_normal([28*28*1,10]))
B3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer2, W3)+B3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

###
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
training_epoch = 5
batch_size = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) #550
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer],feed_dict={X:batch_xs,Y:batch_ys})
        avg_cost += c/total_batch
    print("Epoch :",epoch, 'Cost', avg_cost)

In [ ]:
result = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
result

In [ ]:
r = random.randint(0,mnist.test.num_examples-1)
print('Lbael:', sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print('Lbael:', sess.run(tf.argmax(hypothesis,1),feed_dict={X:mnist.test.images[r:r+1]}))
plt.imshow(
mnist.test.images[r:r+1].reshape(28,28),
    cmap='Greys',
    interpolation = 'nearest')

In [ ]:
X= tf.placeholder(tf.float32, shape=[None,28*28*1])
Y= tf.placeholder(tf.float32, shape=[None,10])

W1 = tf.get_variable("W1",shape=[28*28*1,256],initializer=tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.random_normal([256]))
layer1 = tf.nn.relu(tf.matmul(X,W1)+B1)

W2 = tf.get_variable("W2",shape=[256,256],initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.random_normal([256]))
layer2 = tf.nn.relu(tf.matmul(layer1,W2)+B2)

W3 = tf.get_variable("W3",shape=[256,10],initializer=tf.contrib.layers.xavier_initializer())
B3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer2, W3)+B3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

###
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
training_epoch = 5
batch_size = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) #550
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer],feed_dict={X:batch_xs,Y:batch_ys})
        avg_cost += c/total_batch
    print("Epoch :",epoch, 'Cost', avg_cost)

In [ ]:
result = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
result

# 오버피팅 방지 방법 drop out 방식

In [6]:


X= tf.placeholder(tf.float32, shape=[None,28*28*1])
Y= tf.placeholder(tf.float32, shape=[None,10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1",shape=[28*28*1,256],initializer=tf.contrib.layers.xavier_initializer())
B1 = tf.Variable(tf.random_normal([256]))
layer1 = tf.nn.relu(tf.matmul(X,W1)+B1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)

W2 = tf.get_variable("W2",shape=[256,256],initializer=tf.contrib.layers.xavier_initializer())
B2 = tf.Variable(tf.random_normal([256]))
layer2 = tf.nn.relu(tf.matmul(layer1,W2)+B2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)

W3 = tf.get_variable("W3",shape=[256,10],initializer=tf.contrib.layers.xavier_initializer())
B3 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(layer2, W3)+B3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

###
prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
training_epoch = 5
batch_size = 100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size) #550
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer],feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.7})
        avg_cost += c/total_batch
    print("Epoch :",epoch, 'Cost', avg_cost)

Epoch : 0 Cost 0.41812248323451356
Epoch : 1 Cost 0.2936092029918322
Epoch : 2 Cost 0.2722450764477253
Epoch : 3 Cost 0.28051969552581957
Epoch : 4 Cost 0.26473247682167744


In [13]:
result = sess.run(accuracy, feed_dict={X:x_test, Y:y_test, keep_prob : 1})
result

0.9587

# CNN적용

In [17]:
learning_rate = 0.001
training_epoch = 15


In [ ]:
# CNN  Layer

# 1층 : 3*3 필터 32개 (dropout rate: 0.3 )
# relu-max pooling
# 2층 : 3*3 필터 64개 (dropout rate: 0.3 )
# relu-max pooling
# 3층 : 3*3 필터 128개 (dropout rate: 0.3 )
# relu-max pooling

# DNN(FC)  Layer
# Initializer : xavier initializer
# 1층 : hidden node 625개 (dropout rate: 0.3 )
# activation function : Relu
# 2층 : hidden node 256개 (dropout rate: 0.3 )

# Softmax Layer로 multi label classification
# AdamOptimizer
# learning_rate : 0.001

In [ ]:

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

X_img = tf.reshape(X, [-1, 28, 28, 1])   # 첫 번째가 개수, 나머지는 이미지의 가로/세로/높이
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))   # 필터 개수는 자유롭게 지정 가능(크기, 개수), 1시그마를 0.01로 해서 초기화값의 범위를 줄임
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')   # 심층신경망 통과한 L1의 크기 -1*28*28(same)*32(필터 개수) -> -1 = None
# SAME = stride가 1인 기준으로 이미지가 똑같이 나온다 -> 2는 이미지가 동일한 크기여야 하므로 커널을 통과한 이미지는 1/2 크기가 된다

L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize-[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
# 여기서의 SAME은 zero padding을 하지 않음(padding은 stride가 1일 때를 기준으로 해서 strides가 2면 이미지의 크기가 절반으로 줄어든다
# L1 = -1*14*14*32(max pooling)
# pooling의 커널과 conventional의 필터는 서로 다른 개념이다 ->

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))   # 64만 변경 가능(3*3은 필터 크기가 같고, 32는 이전 커널을 통과한 개수이므로 변경 불가)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")   # L2 = -1*14*14*64 ->

L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')   # L2 = -1 * 7 * 7 * 64

# FC에 넣기 위해서는 펼쳐서 넣어야 한다
L2_flat = tf.reshape(L2, [-1, 7*7*64])
w3 = tf.get_variable("W3", shape=[7*7*64, 10])   # 입력 데이터의 개수 = 7*7*64, 출력 데이터의 개수 = 라벨(0~9)

In [ ]:
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)  
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
        c, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys})
        avg_cost += c/total_batch 
    print("Epoch: ", epoch, "Cost: ", avg_cost) 

In [ ]:
print(sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))